# PROGRAMA DEFINITIVO DE PAGE-SPEED

- Ya esta acabado, solo hay que pasarle la lista con todas las URL's a la funcion Global

- Devuelve un dataframe donde cada linea es una pagina, las variables tienen _m para móvil y _o para ordenador, 
  y también se printan los links que han fallado, estos se podrían volver a revisar con un time.sleep
  mas alto, o comprobar manualmente si estan en PageSpeed
  
- Hay dos módulos Global, el último es para la ejecución en paralelo, pero esta el riesgo de que bloqueen la IP y   el programa de error, si ocurre, utilizar el otro modulo para ejecución normal. (En la prueba con 10 link funcionaba bien)



In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as ec
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

headers = {'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}
def INICIAR_DRIVER():
    options=Options()
    userAgent= 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-notifications')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--no-default-browser-check')
    options.add_argument('--no-first-run')
    options.add_argument('--no-proxy-server')
    options.add_argument('--disable-blink-features=AutomationControlled')
    exp_opt=['enable-automation', 'ignore-certificate-errors','enable-logging']
    options.add_experimental_option('excludeSwitches', exp_opt)
    prefs={'profile.default_content_setting_values.notifications':2, 'intl.accept_languages':['es-ES','es'],'credentials_enable_service':False}
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver

In [2]:
def pagespeed(url):
    
    # Si no se ejecuta en paralelo quitar estas dos lineas de abajo:
    
    t = random.choice([0, 0.25, 0.5, 0.75, 1])
    time.sleep(t)

    driver = INICIAR_DRIVER()
    wait = WebDriverWait(driver, 1.5)    
    path ='https://pagespeed.web.dev'
    driver.get(path)

    text_input = wait.until(ec.visibility_of_element_located((By.XPATH,'//*[@id="i4"]')))
    text_input.send_keys(url)
    buton = wait.until(ec.element_to_be_clickable((By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div[2]/div/div[2]/form/div[2]/button')))
    buton.click()
    
    time.sleep(6)
    
    res = driver.page_source
    soup = BeautifulSoup(res,'html.parser')
    
    driver.quit()
    
    try:
        soups = soup.find_all('div', role='tabpanel')
        
        columnas = ['Link','Tipo','LCP','INP','FCP','TTF','CLS','FID']
        disp = ['Móvil','Ordenador']
        df = pd.DataFrame(columns=columnas)
    
        
        for i,soup in enumerate(soups):
            dic = {'Link':url,'Tipo':disp[i],'LCP':pd.NA,'INP':pd.NA,'FCP':pd.NA,'TTF':pd.NA,'CLS':pd.NA,'FID':pd.NA}
            soup = soup.find('div', style = 'display: none;')
            elementos = soup.find_all('div', class_ = 'PZeSbe')
            for j in elementos:
                var = j.find('span').text
                var = var.split('(')[1][:3]
                value = j.find('div', class_='cOTkPe').text
                if len(value)> 40:
                    value = value.split('.')[2:]
                    value = str(value[0])+'.'+str(value[1])
                    dic[var]= value
                else:
                    dic[var]=value
                    
            df2 = pd.DataFrame([dic])
            df= pd.concat([df, df2], ignore_index=True)
        
        print(f'{GREEN}Link hecho{RESET}')
        return df
    except:
        print(f'{RED}Error en {url}{RESET}')

In [3]:
# ESTE NO ES EN PARALELO, 3 VECES MÁS LENTO, HABRÍA QUE QUITAR LO QUE ESTA COMENTADO EN LA FUNCIÓN ANTERIOR

def Global(lista):
    
    columnas = ['Link','Tipo','LCP','INP','FCP','TTF','CLS','FID']
    df = pd.DataFrame(columns=columnas)
    
    print('EMPEZANDO PROGRAMA:')
    print()
    for i in lista:
        try:
            df2 = pagespeed(i)
            df = pd.concat([df, df2], ignore_index=True)
        except:
            print(f'{RED}Error con el link: {i}{RESET}')

    df_pivot = df.pivot(index='Link', columns='Tipo', values=['LCP', 'INP', 'FCP', 'TTF', 'CLS', 'FID'])
    df_pivot.columns = [f'{metric}_{device[0].lower()}' for metric, device in df_pivot.columns]
    df_pivot.reset_index(inplace=True)
    
    print()
    print('PROGRAMA FINALIZADO')
    return df_pivot

In [4]:
# Esto es para paralelo

import concurrent.futures
import random

def Global(lista):
    
    
    columnas = ['Link','Tipo','LCP','INP','FCP','TTF','CLS','FID']
    df = pd.DataFrame(columns=columnas)
    
    print('EMPEZANDO PROGRAMA:')
    print()
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as exe:
        futuros = []
        for i in lista:
            try:
                futuros.append(exe.submit(pagespeed,i))
            except:
                print(f'{RED}Error con el link: {i}{RESET}')
        for futuro in futuros:
            df2 = futuro.result()
            df = pd.concat([df, df2], ignore_index=True)
    
    df_pivot = df.pivot(index='Link', columns='Tipo', values=['LCP', 'INP', 'FCP', 'TTF', 'CLS', 'FID'])
    df_pivot.columns = [f'{metric}_{device[0].lower()}' for metric, device in df_pivot.columns]
    df_pivot.reset_index(inplace=True)
    
    print()
    print('PROGRAMA FINALIZADO')
    return df_pivot

In [5]:
# ESTE SERÍA EL MAIN, LO HE PROBADO CON UNOS POCOS LINKS

RED = "\033[31m"
GREEN = "\033[32m"
RESET = "\033[0m"
    
lista = ["http://bwin.es/", "https://www.independent.co.uk/sport", "https://www.surfer.com/", "https://www.worldskate.org/", "https://theathletic.com/uk/", "https://www.mlb.com/", "https://worldrowing.com/", "https://www.mmafighting.com/", "https://www.sportico.com/", "https://www.cyclingnews.com/", "https://www.goal.com/", "https://www.businessinsider.es/", "https://racer.com/", "https://www.ufc.com/", "https://sports.yahoo.com/", "http://transfermarkt.es/", "https://www.livefutbol.com/", "https://geniussports.com/", "https://www.triathlete.com/", "https://frontofficesports.com/", "https://www.cricbuzz.com/", "https://www.world.rugby/", "https://www.cricket.com.au/", "http://todocircuito.com/", "https://espndeportes.espn.com/", "https://www.relevo.com/", "https://www.powerlifting.sport/", "https://bleacherreport.com/", "https://www.superdeporte.es/", "https://www.theguardian.com/uk/sport", "https://www.motogp.com/en", "https://www.lequipe.fr/", "https://bwfbadminton.com/", "https://www.skysports.com/", "https://www.drf.com/", "https://www.si.com/", "https://www.bbc.com/sport", "https://www.runnersworld.com/", "https://www.athleticsweekly.com/", "https://as.com/", "https://www.sportcal.com/", "https://www.nbcsports.com/", "https://www.canoeicf.com/", "https://www.triathlon.org/", "https://www.sportspromedia.com/", "https://www.eurosport.es/"]


DF = Global(lista)


EMPEZANDO PROGRAMA:

Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho
Link hecho

PROGRAMA FINALIZADO


In [6]:
DF.head(15)

,Link,LCP_m,LCP_o,INP_m,INP_o,FCP_m,FCP_o,TTF_m,TTF_o,CLS_m,CLS_o,FID_m,FID_o
0,http://bwin.es/,1.9 s,0.9 s,521 ms,157 ms,1.2 s,0.8 s,0.5 s,0.4 s,0.15,0.25,110 ms,14 ms
1,http://todocircuito.com/,2 s,1.3 s,184 ms,97 ms,1.3 s,0.9 s,0.6 s,0.4 s,0.08,0.41,21 ms,4 ms
2,http://transfermarkt.es/,2.5 s,2.1 s,175 ms,62 ms,2 s,1.7 s,1.2 s,1.1 s,0.25,0.14,25 ms,3 ms
3,https://as.com/,1.5 s,1.2 s,268 ms,83 ms,1.2 s,0.7 s,0.3 s,0.2 s,0.03,0.09,49 ms,9 ms
4,https://bleacherreport.com/,2.3 s,2.1 s,180 ms,78 ms,1.5 s,1.2 s,0.7 s,0.5 s,0.19,0.12,22 ms,6 ms
5,https://bwfbadminton.com/,2.9 s,2.4 s,205 ms,48 ms,1.9 s,1.5 s,0.7 s,0.8 s,0.89,0.8,14 ms,2 ms
6,https://espndeportes.espn.com/,4.9 s,4.6 s,442 ms,258 ms,1.9 s,2.3 s,0.7 s,0.8 s,0.08,0.11,47 ms,7 ms
7,https://frontofficesports.com/,1.7 s,1.7 s,226 ms,97 ms,1.4 s,1.4 s,0.6 s,0.8 s,0.07,0.09,60 ms,11 ms
8,https://geniussports.com/,4.5 s,2.7 s,225 ms,63 ms,3.4 s,2.2 s,2.2 s,1.6 s,0.01,0.06,21 ms,3 ms
9,https://racer.com/,1.7 s,1.5 s,95 ms,36 ms,1.2 s,1 s,0.5 s,0.6 s,0.16,0.12,15 ms,3 ms


In [7]:
DF.to_csv('/Users/pablogandia/Desktop/veamos.csv')

# IDEA PROXIES

· La idea sería ver cuantas proxies públicas acepta la página, esto puede ser una medida de seguridad o algo así. 



· Hay horas que mucha gente utiliza estas proxies públicas y se saturan, haciendo que funcionen muy pocas, es    decir, la hora en que se ejecuta podría crear un sesgo en esta variable, pero solo es una propuesta.

In [ ]:
import requests as res
import time
import concurrent.futures
from bs4 import BeautifulSoup

RED = "\033[31m"
GREEN = "\033[32m"
RESET = "\033[0m"
headers = {'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}

def proxyscrape1():
    url = 'https://api.proxyscrape.com/v3/free-proxy-list/get?request=getproxies&proxy_format=protocolipport&format=json'
    r = res.get(url, headers = headers, timeout = 10)
    datos = r.json()
    http_prox = []
    for i in datos['proxies']:
        if i['protocol']=='http' and i['anonymity']=='elite':
            proxy = f"http://{i['ip']}:{i['port']}"
            http_prox.append(proxy)
    return http_prox

def proxyscrape2():
    url = 'https://free-proxy-list.net'
    r = res.get(url, headers = headers, timeout = 10)
    soup = BeautifulSoup(r.text, 'html.parser')
    filas = soup.find('tbody').find_all('tr')
    http_prox = []
    for i in filas:
        params = i.find_all('td')
        ip = params[0].text
        puerto = params[1].text
        tipo = params[4].text
        google = params[5].text
        if tipo == 'elite proxy' and google == 'yes':
            proxy = f'http://{ip}:{puerto}'
            http_prox.append(proxy)
    return http_prox

def prove_prx(proxy, url):
    dic = {'proxy':proxy, 'tiempo':None,'res':None}
    try:
        proxies = {'http':proxy, 'https':proxy}
        inicio = time.time()
        r = res.get(url, headers = headers,proxies=proxies,timeout = 10)
        dic['tiempo']=time.time()-inicio
        dic['res']=r.status_code
    except:
        pass
    return dic

def bucle(proxies,url):
    yes = 0
    no = 0
    min_time = 9999
    with concurrent.futures.ThreadPoolExecutor() as exe:
        futuros = []
        for proxy in proxies:
            futuros.append(exe.submit(prove_prx,proxy,url))
            
        for futuro in concurrent.futures.as_completed(futuros):
            dic = futuro.result()
            if dic['res'] == 200:
                yes +=1
                print(f"{GREEN}Proxy {dic['proxy']} funciona --> {dic['tiempo']}s{RESET}")
                if dic['tiempo']<min_time:
                    min_time = dic['tiempo']
            else:
                print(f"{RED}Proxy {dic['proxy']} no funciona.{RESET}")
                no += 1
    if min_time == 9999:
        min_time = None
    tupla = (yes,no,min_time)
    return tupla

url = 'https://www.n3xtsports.com'
lista_prx = proxyscrape1()
lista_prx.extend(proxyscrape2())
lista_prx = list(set(lista_prx))
tupla = bucle(lista_prx,url)
print(tupla)

Proxy http://185.217.199.7:4444 no funciona.
Proxy http://185.105.91.183:4444 no funciona.
Proxy http://84.252.74.49:4444 no funciona.
Proxy http://185.128.107.15:4444 no funciona.
Proxy http://67.227.240.29:3128 no funciona.
Proxy http://43.255.113.232:8080 no funciona.
Proxy http://125.77.25.178:8080 no funciona.
Proxy http://185.105.91.151:4444 no funciona.
Proxy http://2.189.59.122:3128 no funciona.
Proxy http://67.227.240.22:3128 no funciona.
Proxy http://185.105.89.143:4444 no funciona.
Proxy http://185.217.199.54:4444 no funciona.
Proxy http://209.97.150.167:8080 no funciona.
Proxy http://185.128.106.151:4444 no funciona.
Proxy http://72.10.164.178:28211 no funciona.
Proxy http://67.43.227.227:1779 no funciona.
Proxy http://85.209.153.174:4153 no funciona.
Proxy http://72.10.160.173:27063 no funciona.
Proxy http://85.209.153.174:5678 no funciona.
Proxy http://178.128.200.87:80 no funciona.
Proxy http://185.128.106.55:4444 no funciona.
Proxy http://103.75.117.85:80 no funciona.
P